<a href="https://www.kaggle.com/code/velimirkovacic/rank-0036533917-data-cleaning-fe-and-extratrees?scriptVersionId=183361277" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Rank Notebook

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

## 1. Data Analysis
### 1.1. Dimensionality of the dataset 


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df_train = pd.read_csv("../input/dapprojekt24-1/train.csv")
df_test = pd.read_csv("../input/dapprojekt24-1/test.csv")

print("Train set")
print("Shape:", df_train.shape)

print("Column names: ", end="")
for c in df_train.columns:
    print(c, end= ", ")

print("\nUnique stocks:", df_train["Symbol"].nunique())

print("\n")

print("Test set")
print("Shape:", df_test.shape)

print("Column names: ", end="")
for c in df_test.columns:
    print(c, end= ", ")

print("\nUnique stocks:", df_test["Symbol"].nunique())


if(set(df_test["Symbol"].unique()) == set(df_train["Symbol"].unique())):
    print("\nUnique stocks are the same among both datasets")


df_train

/kaggle/input/dapprojekt24-1/train.csv
/kaggle/input/dapprojekt24-1/test.csv
Train set
Shape: (501400, 10)
Column names: Date, Symbol, Adj Close, Close, High, Low, Open, Volume, Target, Id, 
Unique stocks: 200


Test set
Shape: (209600, 9)
Column names: Date, Symbol, Adj Close, Close, High, Low, Open, Volume, Id, 
Unique stocks: 200

Unique stocks are the same among both datasets


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Target,Id
0,2010-01-04,MMM,53.295380,83.019997,83.449997,82.669998,83.089996,3043700.0,0,0
1,2010-01-05,MMM,52.961575,82.500000,83.230003,81.699997,82.800003,2847000.0,0,1
2,2010-01-06,MMM,53.712681,83.669998,84.599998,83.510002,83.879997,5268500.0,0,2
3,2010-01-07,MMM,53.751179,83.730003,83.760002,82.120003,83.320000,4470100.0,0,3
4,2010-01-08,MMM,54.129955,84.320000,84.320000,83.300003,83.690002,3405800.0,0,4
...,...,...,...,...,...,...,...,...,...,...
501395,2019-12-24,ZTS,131.227859,132.919998,133.080002,131.289993,132.210007,442500.0,1,501395
501396,2019-12-26,ZTS,131.336456,133.029999,133.190002,132.320007,133.190002,929400.0,1,501396
501397,2019-12-27,ZTS,131.553665,133.250000,133.479996,132.380005,133.399994,1296100.0,1,501397
501398,2019-12-30,ZTS,130.526886,132.210007,133.660004,131.809998,133.570007,942000.0,1,501398


There are two datasets, a train set and a test set. The train set consists of 501400 samples and 10 features. The test set consists of 209600 samples and 9 features (missing the target feature). 

The column (feature) names are: 
* Date
* Symbol
* Adj Close
* Close
* High
* Low
* Open
* Volume
* Target
* Id

There are 200 different stocks, denoted by their symbol in the Symbol column. They are the same among both datasets.

### 1.2. Missing values

In [3]:
def find_missing_sections(df):
    rows_with_missing_values = df.isnull().sum(axis = 1) > 0

    missing_value_indexes = df[rows_with_missing_values].index

    if (len(missing_value_indexes) == 0):
        return []

    # We find the beginning and and indexes of sections with missing values
    missing_value_limits = [missing_value_indexes[0]]
    for i in range(len(missing_value_indexes[1:])):
        if missing_value_indexes[i] - 1 > missing_value_indexes[i - 1]:
            missing_value_limits += [missing_value_indexes[i - 1], missing_value_indexes[i]]
    missing_value_limits += [missing_value_indexes[-1]] 


    # We transform them into (section beginning, section end) pairs
    missing_value_sections = []
    for i in range(0, len(missing_value_limits), 2):
        missing_value_sections += [(missing_value_limits[i], missing_value_limits[i + 1])]

    return missing_value_sections


In [4]:
print("Train")

missing_value_sections_train = find_missing_sections(df_train)
print("Sections with missing values: ", len(missing_value_sections_train))
for section in missing_value_sections_train:
    print(section, "length:", section[1] - section[0])


print("Test")

missing_value_sections_test = find_missing_sections(df_test)
print("Sections with missing values: ", len(missing_value_sections_test))
for section in missing_value_sections_test:
    print(section, "length:", section[1] - section[0])

Train
Sections with missing values:  28
(500, 515) length: 15
(17549, 18524) length: 975
(50140, 51252) length: 1112
(85238, 85349) length: 111
(90252, 90252) length: 0
(100280, 101468) length: 1188
(117829, 120182) length: 2353
(132871, 133571) length: 700
(155434, 155998) length: 564
(188025, 188264) length: 239
(190532, 192158) length: 1626
(193039, 195340) length: 2301
(200560, 203066) length: 2506
(205574, 205600) length: 26
(208081, 208302) length: 221
(228137, 229129) length: 992
(255714, 257486) length: 1772
(258221, 259062) length: 841
(263235, 265741) length: 2506
(270756, 271962) length: 1206
(278277, 278556) length: 279
(293319, 293397) length: 78
(295826, 296197) length: 371
(358501, 359073) length: 572
(418669, 419819) length: 1150
(421176, 421409) length: 233
(456274, 458780) length: 2506
(498893, 499667) length: 774
Test
Sections with missing values:  3
(83840, 84584) length: 744
(110040, 110879) length: 839
(190736, 191680) length: 944


There are 27245 rows with one or more missing values in the train set organized in 28 distinct sections. 
* 500 - 515, Only the Volume value is missing. (Replace)
* 17549 - 18524, a section of 975 days with no values except the date and stock symbol (Erase)
* 50140 - 51252, a section of 1112 days with no values except the date and stock symbol (Erase)
* and so on...

For each section we can either replace the missing values or erase the rows. For the section with 15 missing Volume values it would make sense to just replace the value as it is a rather short section and this will not distort the data too much. For the very long sections which are missing every value we have no other choice than to erase them as replacing them would not give us any useful information in the future analysis of the data.

We will erase all rows which have no values except the date and stock symbol and search for missing values again. We will not modify the test dataset as we will still have to make predictions even for the samples with missing values.


In [5]:
print("Train")
df_train = df_train.dropna(thresh=8)
missing_value_sections_train = find_missing_sections(df_train)
print("Sections with missing values: ", len(missing_value_sections_train))
for section in missing_value_sections_train:
    print(section, "length:", section[1] - section[0])


Train
Sections with missing values:  1
(500, 515) length: 15


We are still left with the section of 16 rows missing the value for Volume. We will solve this by using linear interpolation to bridge the gap between row 499 and row 516.

In [6]:
missing_begin, missing_end = missing_value_sections_train[0]
df_train.loc[:, "Volume"] = df_train["Volume"].interpolate(method='linear')
result_df = df_train.iloc[range(missing_begin - 1, missing_end + 2)]

In [7]:
print("Train")
df_train = df_train.dropna(thresh=8)
missing_value_sections_train = find_missing_sections(df_train)
print("Sections with missing values: ", len(missing_value_sections_train))
for section in missing_value_sections_train:
    print(section, "length:", section[1] - section[0])

Train
Sections with missing values:  0


There are no more rows with missing values.

### 1.3. Duplicates

In [8]:
print("Train")
duplicate_rows = df_train.duplicated()
print("Numer of duplicate rows:", len(df_train[duplicate_rows]))
duplicate_date_symbol = df_train.duplicated(subset=["Date", "Symbol"])
print("Numer of duplicate Date-Symbol pairs:", len(df_train[duplicate_date_symbol]))
duplicate_id = df_train.duplicated(subset=["Id"])
print("Numer of duplicate Id-s:", len(df_train[duplicate_id]))

print("Test")
duplicate_rows = df_test.duplicated()
print("Numer of duplicate rows:", len(df_test[duplicate_rows]))
duplicate_date_symbol = df_test.duplicated(subset=["Date", "Symbol"])
print("Numer of duplicate Date-Symbol pairs:", len(df_test[duplicate_date_symbol]))
duplicate_id = df_test.duplicated(subset=["Id"])
print("Numer of duplicate Id-s:", len(df_test[duplicate_id]))

Train
Numer of duplicate rows: 0
Numer of duplicate Date-Symbol pairs: 0
Numer of duplicate Id-s: 0
Test
Numer of duplicate rows: 0
Numer of duplicate Date-Symbol pairs: 0
Numer of duplicate Id-s: 0


There is no duplicate data.

### 1.4. Data validation

The following must be true for all samples:
1. Adjusted closing price must be less or equal to the closing price.
2. Low price must be less or equal to the high price.
3. Opening and closing price must be in the interval between the low price and the high price.
4. Volume is an integer.
5. All values are positive.

In terms of equations:

* Adj Close &le; Close
* Low &le; High
* Open ∈ [Low, High]
* Close ∈ [Low, High]
* Volume ∈ $\mathbb{Z}$

In [9]:
def validate_data(df):
    print("Train")
    adjc_bigger_c = df["Adj Close"] > df["Close"]
    print("Rows with Adj Close > Close:", len(df[adjc_bigger_c]))
    low_high = df["Low"] > df["High"]
    print("Rows with Low > High:", len(df[low_high]))
    open_in = (df["Open"] < df["Low"]) | (df["Open"] > df["High"])
    print("Rows with Open < Low or Open > High:", len(df[open_in]))
    close_in = ( df["Close"] < df["Low"]) | (df["Close"] > df["High"])
    print("Rows with Close < Low or Close > High:", len(df[close_in]))
    negative = (df["Adj Close"] < 0) | (df["Close"] < 0) | (df["Open"] < 0) | (df["Low"] < 0) | (df["High"] < 0)
    print("Rows with negative features:", len(df[negative]))


    print("Are they the same? ", end="")
    print((df[negative].index == df[open_in].index).all() and (df[low_high].index == df[open_in].index).all() and (df[close_in].index == df[open_in].index).all())

    isnt_Z = df["Volume"].apply(lambda x: not x.is_integer())
    print("Rows where Volume isn\'t an integer:", len(df[isnt_Z]))

    return negative, isnt_Z


issue1, issue2 = validate_data(df_train)
df_train[issue1]


Train
Rows with Adj Close > Close: 0
Rows with Low > High: 110
Rows with Open < Low or Open > High: 110
Rows with Close < Low or Close > High: 110
Rows with negative features: 110
Are they the same? True
Rows where Volume isn't an integer: 16


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Target,Id
60596,2011-09-14,BALL,17.094999,17.094999,-51.885000,16.750000,17.125000,2850400.0,1,60596
60597,2011-09-15,BALL,17.115000,17.115000,-51.750000,17.020000,17.240000,3416400.0,1,60597
60598,2011-09-16,BALL,17.155001,17.155001,-51.764997,16.985001,17.165001,3888200.0,1,60598
60599,2011-09-19,BALL,16.855000,16.855000,-50.774998,16.565001,16.885000,2524400.0,1,60599
60600,2011-09-20,BALL,16.754999,16.754999,-51.270000,16.705000,16.879999,2251000.0,1,60600
...,...,...,...,...,...,...,...,...,...,...
60701,2012-02-14,BALL,19.934999,19.934999,-59.909998,19.805000,19.915001,1077600.0,1,60701
60702,2012-02-15,BALL,19.865000,19.865000,-60.105000,19.820000,19.959999,1644200.0,1,60702
60703,2012-02-16,BALL,20.160000,20.160000,-60.584999,19.780001,19.885000,1473200.0,1,60703
60704,2012-02-17,BALL,20.055000,20.055000,-60.750000,20.000000,20.250000,1387600.0,1,60704


In [10]:
df_train[issue2]

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Target,Id
500,2011-12-27,MMM,55.581924,82.279999,82.639999,81.620003,81.910004,2.129282e+06,1,500
501,2011-12-28,MMM,54.818573,81.150002,82.440002,80.940002,82.220001,2.216365e+06,1,501
502,2011-12-29,MMM,55.467079,82.110001,82.199997,81.190002,81.370003,2.303447e+06,1,502
503,2011-12-30,MMM,55.210377,81.730003,82.449997,81.599998,82.230003,2.390529e+06,1,503
504,2012-01-03,MMM,56.399281,83.489998,84.440002,83.360001,83.760002,2.477612e+06,1,504
505,2012-01-04,MMM,56.865433,84.180000,84.260002,83.110001,83.129997,2.564694e+06,1,505
506,2012-01-05,MMM,56.608723,83.800003,83.870003,82.699997,83.529999,2.651776e+06,1,506
507,2012-01-06,MMM,56.318230,83.370003,84.239998,83.320000,83.779999,2.738859e+06,1,507
508,2012-01-09,MMM,56.655991,83.870003,84.059998,83.260002,83.580002,2.825941e+06,1,508
509,2012-01-10,MMM,56.946457,84.300003,85.190002,84.040001,84.589996,2.913024e+06,1,509


There are 110 samples where the High price is negative and about three times higher than the other associated price values. We suspect that the High price is not random but somehow modified from the original price. We can try dividing it by -3 and seeing if the aforementioned constraints are satisfied.

There are also the 16 samples on which we used linear interpolation to fill the gap, we will simply round these to satify the integer constraint.

In [11]:
df_train.loc[issue1, "High"] /= (-3)
df_train.loc[issue2, "Volume"] = df_train.loc[issue2, "Volume"].round()

_, _ = validate_data(df_train)


Train
Rows with Adj Close > Close: 0
Rows with Low > High: 0
Rows with Open < Low or Open > High: 0
Rows with Close < Low or Close > High: 0
Rows with negative features: 0
Are they the same? True
Rows where Volume isn't an integer: 0


Dividing the High price by -3 worked very well, we also tried using a smaller value to divide by such as -3.1, this yielded unfavorable results as not all rows satisfied the constraints. This proces that -3 is the smallest negative value we can use in order to "fix" the prices.

In [12]:
_, _ = validate_data(df_test)


Train
Rows with Adj Close > Close: 0
Rows with Low > High: 0
Rows with Open < Low or Open > High: 0
Rows with Close < Low or Close > High: 0
Rows with negative features: 0
Are they the same? True
Rows where Volume isn't an integer: 2530


There are no problematic rows in the test set.

In [13]:
df_train.to_csv("train2.csv", index=None)

## 2. Feature engineering


In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
import shap
from IPython.display import display


First we will import the train dataset created in the first notebook. 

In [15]:

df_train = pd.read_csv("train2.csv")
df_train['Date'] = pd.to_datetime(df_train['Date'])

df_test = pd.read_csv("../input/dapprojekt24-1/test.csv")



### 2.1. Domain specific features
#### 2.1.1 Simple Moving Average SMA
SMA is calculated as the average closing price of the stock last N days. We will use N = 200 as it is the most commomn within trading circles. The 199 NaNs per stock will be filled with the Close values.

In [16]:
df_train['SMA'] = df_train.groupby('Symbol')['Close'].rolling(window=200).mean().reset_index(0, drop=True)
df_train.loc[df_train['SMA'].isna(), 'SMA'] = df_train['Close']


#### 2.1.2 Exponential Moving Average (EMA)
Unlike SMA, EMA places greater importance on more recent values. It is a weighted average. We will use a 12 day span.

In [17]:
df_train['EMA'] = df_train.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=12, adjust=False).mean()).reset_index(0, drop=True)

#### 2.1.3 Moving Average Convergence Divergence (MACD)
MACD is the difference between a 12 day span EMA and a 26 day span EMA. 

In [18]:
df_train['MACD'] = df_train['EMA'] - df_train.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=26, adjust=False).mean()).reset_index(0, drop=True)

#### 2.1.4 Relative Strength Index (RSI)
RSI is a an indicator within range of 0 to 100 and is calculated as: 
$RSI = 100 - \frac{100}{1 + \frac{AvgGain}{AvgLoss}}.$ We will use a 14 day span.

The missing values will be filled with the mean value of that stock. We will do this for all other instances of this issue.



In [19]:
change = df_train.groupby('Symbol')['Close'].diff()

df_train['Gain'] = change.apply(lambda x: x if x > 0 else 0)
df_train['Loss'] = change.apply(lambda x: -x if x < 0 else 0)

avgGain = df_train.groupby('Symbol')['Gain'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)
avgLoss = df_train.groupby('Symbol')['Loss'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)


df_train['RSI'] = 100 - (100 / (1 + avgGain/avgLoss))
df_train.loc[df_train['Loss'] < 1e-2, 'RSI'] = 100
df_train["RSI"] = df_train.groupby('Symbol')['RSI'].transform(lambda x: x.fillna(x.mean()))
df_train = df_train.drop(columns=['Gain', 'Loss'])

#### 2.1.5 On-Balance Volume (OBV)
OBV is calculated as:

$
OBV_i = OBV_{i -1} \begin{cases} 
Volume_i & \text{if } Close_i > Close_{i-1} \\
0 & \text{if } Close_i = Close_{i-1} \\
-Volume_i & \text{if } Close_i < Close_{i-1} 
\end{cases}
$

In [20]:
df_train['Dir'] = change.apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_train['OBV'] = df_train.groupby('Symbol')['Volume'].transform(lambda x: x * df_train['Dir']).cumsum()

df_train = df_train.drop(columns=['Dir'])

In [21]:
df_train

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Target,Id,SMA,EMA,MACD,RSI,OBV
0,2010-01-04,MMM,53.295380,83.019997,83.449997,82.669998,83.089996,3043700.0,0,0,83.019997,83.019997,0.000000,100.000000,0.000000e+00
1,2010-01-05,MMM,52.961575,82.500000,83.230003,81.699997,82.800003,2847000.0,0,1,82.500000,82.939997,-0.041481,0.000000,-2.847000e+06
2,2010-01-06,MMM,53.712681,83.669998,84.599998,83.510002,83.879997,5268500.0,0,2,83.669998,83.052305,0.019825,100.000000,2.421500e+06
3,2010-01-07,MMM,53.751179,83.730003,83.760002,82.120003,83.320000,4470100.0,0,3,83.730003,83.156566,0.072418,100.000000,6.891600e+06
4,2010-01-08,MMM,54.129955,84.320000,84.320000,83.300003,83.690002,3405800.0,0,4,84.320000,83.335556,0.159863,100.000000,1.029740e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474166,2019-12-24,ZTS,131.227859,132.919998,133.080002,131.289993,132.210007,442500.0,1,501395,115.112650,127.882202,2.712151,100.000000,1.010496e+11
474167,2019-12-26,ZTS,131.336456,133.029999,133.190002,132.320007,133.190002,929400.0,1,501396,115.293150,128.674170,2.921901,100.000000,1.010506e+11
474168,2019-12-27,ZTS,131.553665,133.250000,133.479996,132.380005,133.399994,1296100.0,1,501397,115.478400,129.378144,3.070488,100.000000,1.010519e+11
474169,2019-12-30,ZTS,130.526886,132.210007,133.660004,131.809998,133.570007,942000.0,1,501398,115.654900,129.813815,3.068948,89.496454,1.010509e+11


### 2.2. History related features


Here we will invent new history related features that may help in our predictions.

#### 2.2.1 Close Entropy (CE)
We will calculate entropy of closing prices. We will use a 10 day span. Missing entropies will be filled by the mean of the specific stock.

In [22]:
probs = df_train.groupby('Symbol')['Close'].transform(lambda x: x / x.sum())

df_train["CE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x * np.log2(x))).reset_index(drop=True)
df_train["CE"] = df_train.groupby('Symbol')['CE'].transform(lambda x: x.fillna(x.mean()))

#### 2.2.2 Average Open-Close (AOC)
This will be the average value of Open - Close. We will use a 20 day span.

In [23]:
open_mean = df_train.groupby('Symbol')['Open'].rolling(window=20).mean().reset_index(drop=True)
close_mean = df_train.groupby('Symbol')['Close'].rolling(window=20).mean().reset_index(drop=True)

df_train["AOC"] = open_mean - close_mean
df_train["AOC"] = df_train.groupby('Symbol')['AOC'].transform(lambda x: x.fillna(x.mean()))

#### 2.2.3 Close Variance (CV)
This will be the variance of Close. We will use a 30 day span.


In [24]:
df_train["CV"] = df_train.groupby('Symbol')["Close"].rolling(window=30).var().reset_index(drop=True)
df_train["CV"] = df_train.groupby('Symbol')["CV"].transform(lambda x: x.fillna(x.mean()))

#### 2.2.4 Volume Variance (VV)
This is the variance of Volume. We will use a 30 day span.

In [25]:
df_train["VV"] = df_train.groupby('Symbol')["Volume"].rolling(window=30).var().reset_index(drop=True)
df_train["VV"] = df_train.groupby('Symbol')["VV"].transform(lambda x: x.fillna(x.mean()))

#### 2.2.5 Average High-Low (AHL)
This will be the average value of High - Low. We will use a 20 day span.

In [26]:
high_mean = df_train.groupby('Symbol')['High'].rolling(window=20).mean().reset_index(drop=True)
low_mean = df_train.groupby('Symbol')['Low'].rolling(window=20).mean().reset_index(drop=True)

df_train["AHL"] = high_mean - low_mean
df_train["AHL"] = df_train.groupby('Symbol')['AHL'].transform(lambda x: x.fillna(x.mean()))

#### 2.2.6 High Variance (HV)
This is the variance of High. We will use a 20 day span.

In [27]:
df_train["HV"] = df_train.groupby('Symbol')['High'].rolling(window=20).var().reset_index(drop=True)
df_train["HV"] = df_train.groupby('Symbol')['HV'].transform(lambda x: x.fillna(x.mean()))

#### 2.2.7 Open Entropy (OE)
This is the entropy of Open. We will use a 10 day span.

In [28]:
probs = df_train.groupby('Symbol')['Open'].transform(lambda x: x / x.sum())

df_train["OE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x/np.sum(x) * np.log2(x/np.sum(x)))).reset_index(drop=True)
df_train["OE"] = df_train.groupby('Symbol')['OE'].transform(lambda x: x.fillna(x.mean()))

In [29]:
df_train

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,Target,Id,...,MACD,RSI,OBV,CE,AOC,CV,VV,AHL,HV,OE
0,2010-01-04,MMM,53.295380,83.019997,83.449997,82.669998,83.089996,3043700.0,0,0,...,0.000000,100.000000,0.000000e+00,0.044735,-0.010517,2.329259,2.749234e+12,0.841072,1.509042,3.321765
1,2010-01-05,MMM,52.961575,82.500000,83.230003,81.699997,82.800003,2847000.0,0,1,...,-0.041481,0.000000,-2.847000e+06,0.044735,-0.010517,2.329259,2.749234e+12,0.841072,1.509042,3.321765
2,2010-01-06,MMM,53.712681,83.669998,84.599998,83.510002,83.879997,5268500.0,0,2,...,0.019825,100.000000,2.421500e+06,0.044735,-0.010517,2.329259,2.749234e+12,0.841072,1.509042,3.321765
3,2010-01-07,MMM,53.751179,83.730003,83.760002,82.120003,83.320000,4470100.0,0,3,...,0.072418,100.000000,6.891600e+06,0.044735,-0.010517,2.329259,2.749234e+12,0.841072,1.509042,3.321765
4,2010-01-08,MMM,54.129955,84.320000,84.320000,83.300003,83.690002,3405800.0,0,4,...,0.159863,100.000000,1.029740e+07,0.044735,-0.010517,2.329259,2.749234e+12,0.841072,1.509042,3.321765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474166,2019-12-24,ZTS,131.227859,132.919998,133.080002,131.289993,132.210007,442500.0,1,501395,...,2.712151,100.000000,1.010496e+11,0.119094,-0.423498,21.130307,8.145555e+11,1.837501,18.013597,3.321464
474167,2019-12-26,ZTS,131.336456,133.029999,133.190002,132.320007,133.190002,929400.0,1,501396,...,2.921901,100.000000,1.010506e+11,0.119843,-0.366498,23.631264,8.651257e+11,1.793501,20.652378,3.321424
474168,2019-12-27,ZTS,131.553665,133.250000,133.479996,132.380005,133.399994,1296100.0,1,501397,...,3.070488,100.000000,1.010519e+11,0.120549,-0.384499,25.348339,8.832135e+11,1.749501,23.210673,3.321439
474169,2019-12-30,ZTS,130.526886,132.210007,133.660004,131.809998,133.570007,942000.0,1,501398,...,3.068948,89.496454,1.010509e+11,0.121163,-0.354999,25.696162,9.263036e+11,1.756001,24.807829,3.321516


### 2.3 Normalization

We will create a new dataset with the 12 new features and then normalize them with Min-Max normalization.

In [30]:
df = df_train.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Id", "Date", "Volume"])

for i in df.columns:
    if i == 'Symbol':
        continue
    df[i] = df.groupby('Symbol')[i].transform(lambda x: ((x - x.min() )/ (x.max() - x.min()))).reset_index(drop=True)

df = df.drop(columns=['Symbol'])

In [31]:
df

,Target,SMA,EMA,MACD,RSI,OBV,CE,AOC,CV,VV,AHL,HV,OE
0,0.0,0.055388,0.050916,0.629640,1.000000,0.162438,0.411207,0.504970,0.094291,0.115948,0.298412,0.069933,0.951700
1,0.0,0.051910,0.050461,0.626984,0.000000,0.155180,0.411207,0.504970,0.094291,0.115948,0.298412,0.069933,0.951700
2,0.0,0.059736,0.051100,0.630909,1.000000,0.168611,0.411207,0.504970,0.094291,0.115948,0.298412,0.069933,0.951700
3,0.0,0.060138,0.051694,0.634276,1.000000,0.180008,0.411207,0.504970,0.094291,0.115948,0.298412,0.069933,0.951700
4,0.0,0.064085,0.052713,0.639874,1.000000,0.188690,0.411207,0.504970,0.094291,0.115948,0.298412,0.069933,0.951700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474166,1.0,0.991749,0.977097,0.899051,1.000000,0.991833,0.971485,0.094497,0.799234,0.004799,0.650734,0.725616,0.996008
474167,1.0,0.993832,0.984910,0.934832,1.000000,0.994980,0.979917,0.130768,0.894072,0.005157,0.632038,0.832027,0.995658
474168,1.0,0.995969,0.991854,0.960180,1.000000,0.999368,0.987854,0.119314,0.959185,0.005286,0.613342,0.935193,0.995789
474169,1.0,0.998005,0.996151,0.959917,0.877245,0.996179,0.994763,0.138086,0.972374,0.005591,0.616104,0.999600,0.996456


In [32]:
X = df.drop(columns=["Target"])
y = df["Target"]


## 3. Submitting to Competition

### 3.1. Test Set Features


We will have to create all the features for the test set and then normalize them.

In [33]:
df_test['SMA'] = df_test.groupby('Symbol')['Close'].rolling(window=200).mean().reset_index(0, drop=True)
df_test.loc[df_test['SMA'].isna(), 'SMA'] = df_test['Close']
df_test['EMA'] = df_test.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=12, adjust=False).mean()).reset_index(0, drop=True)
df_test['MACD'] = df_test['EMA'] - df_test.groupby('Symbol')['Close'].apply(lambda x: x.ewm(span=26, adjust=False).mean()).reset_index(0, drop=True)
change = df_test.groupby('Symbol')['Close'].diff()
df_test['Gain'] = change.apply(lambda x: x if x > 0 else 0)
df_test['Loss'] = change.apply(lambda x: -x if x < 0 else 0)
avgGain = df_test.groupby('Symbol')['Gain'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)
avgLoss = df_test.groupby('Symbol')['Loss'].rolling(window=14, min_periods=1).mean().reset_index(drop=True)
df_test['RSI'] = 100 - (100 / (1 + avgGain/avgLoss))
df_test.loc[df_test['Loss'] < 1e-2, 'RSI'] = 100
df_test["RSI"] = df_test.groupby('Symbol')['RSI'].transform(lambda x: x.fillna(x.mean()))
df_test = df_test.drop(columns=['Gain', 'Loss'])
df_test['Dir'] = change.apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_test['OBV'] = df_test.groupby('Symbol')['Volume'].transform(lambda x: x * df_test['Dir']).cumsum()
df_test = df_test.drop(columns=['Dir'])
probs = df_test.groupby('Symbol')['Close'].transform(lambda x: x / x.sum())
df_test["CE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x * np.log2(x))).reset_index(drop=True)
df_test["CE"] = df_test.groupby('Symbol')['CE'].transform(lambda x: x.fillna(x.mean()))
open_mean = df_test.groupby('Symbol')['Open'].rolling(window=20).mean().reset_index(drop=True)
close_mean = df_test.groupby('Symbol')['Close'].rolling(window=20).mean().reset_index(drop=True)
df_test["AOC"] = open_mean - close_mean
df_test["AOC"] = df_test.groupby('Symbol')['AOC'].transform(lambda x: x.fillna(x.mean()))
df_test["CV"] = df_test.groupby('Symbol')["Close"].rolling(window=30).var().reset_index(drop=True)
df_test["CV"] = df_test.groupby('Symbol')["CV"].transform(lambda x: x.fillna(x.mean()))
df_test["VV"] = df_test.groupby('Symbol')["Volume"].rolling(window=30).var().reset_index(drop=True)
df_test["VV"] = df_test.groupby('Symbol')["VV"].transform(lambda x: x.fillna(x.mean()))
high_mean = df_test.groupby('Symbol')['High'].rolling(window=20).mean().reset_index(drop=True)
low_mean = df_test.groupby('Symbol')['Low'].rolling(window=20).mean().reset_index(drop=True)
df_test["AHL"] = high_mean - low_mean
df_test["AHL"] = df_test.groupby('Symbol')['AHL'].transform(lambda x: x.fillna(x.mean()))
df_test["HV"] = df_test.groupby('Symbol')['High'].rolling(window=20).var().reset_index(drop=True)
df_test["HV"] = df_test.groupby('Symbol')['HV'].transform(lambda x: x.fillna(x.mean()))

probs = df_test.groupby('Symbol')['Open'].transform(lambda x: x / x.sum())
df_test["OE"] = probs.rolling(window=10).apply(lambda x: -np.sum(x/np.sum(x) * np.log2(x/np.sum(x)))).reset_index(drop=True)
df_test["OE"] = df_test.groupby('Symbol')['OE'].transform(lambda x: x.fillna(x.mean()))


In [34]:
df_test = df_test.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Date", "Volume"])

for i in df_test.columns:
    if i == 'Symbol' or i == 'Id':
        continue
    df_test[i] = df_test.groupby('Symbol')[i].transform(lambda x: ((x - x.min() )/ (x.max() - x.min()))).reset_index(drop=True)

df_test = df_test.drop(columns=['Symbol'])

X_test = df_test.drop(columns=['Id'])
X_test = X_test.fillna(X_test.mean())

### 3.2. Model

And finally train the model which has proven to work best for us: ExtraTreesClassifier and predict the test set.

In [35]:
model = ExtraTreesClassifier()
model.fit(X, y)
y_test = model.predict(X_test)


### 3.3. Submission

In [36]:
submission = df_test.loc[:,df_test.columns.isin(('Id', ))]
submission.loc[:,'Target'] = y_test
submission.to_csv("submission.csv", index=None)
submission


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Id,Target
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0
...,...,...
209595,209595,1.0
209596,209596,1.0
209597,209597,1.0
209598,209598,1.0
